In [5]:
#install.packages("sjmisc")
#install.packages("fdth", quiet=TRUE)

library(readxl)
library(sjmisc)
library(fdth)

Warning message:
"package 'sjmisc' was built under R version 3.6.3"

ERROR: Error in library(fdth): there is no package called 'fdth'
